## JuliaBUGS

In [1]:
using Revise
using Graphs, JuliaBUGS, Distributions
using JuliaBUGS:
    CollectVariables,
    DependencyGraph,
    NodeFunctions,
    ArrayElement,
    ArraySlice,
    ArrayVariable,
    program!

[ Info: Precompiling JuliaBUGS [ba9fb4c0-828e-4473-b6a1-cd2560fee5bf]


In [12]:
model_def = @bugsast begin
    for i in 1:N
        for j in 1:T
            Y[i, j] ~ dnorm(mu[i, j], tau_c)
            mu[i, j] = alpha[i] + beta[i] * (x[j] - xbar)
        end
        alpha[i] ~ dnorm(alpha_c, alpha_tau)
        beta[i] ~ dnorm(beta_c, beta_tau)
    end
    tau_c ~ dgamma(0.001, 0.001)
    sigma = 1 / sqrt(tau_c)
    alpha_c ~ dnorm(0.0, 1.0E-6)
    alpha_tau ~ dgamma(0.001, 0.001)
    beta_c ~ dnorm(0.0, 1.0E-6)
    beta_tau ~ dgamma(0.001, 0.001)
    alpha0 = alpha_c - xbar * beta_c
end

## data
x = [8.0, 15.0, 22.0, 29.0, 36.0]
xbar = 22
N = 30
T = 5
Y = [
    151 199 246 283 320
    145 199 249 293 354
    147 214 263 312 328
    155 200 237 272 297
    135 188 230 280 323
    159 210 252 298 331
    141 189 231 275 305
    159 201 248 297 338
    177 236 285 350 376
    134 182 220 260 296
    160 208 261 313 352
    143 188 220 273 314
    154 200 244 289 325
    171 221 270 326 358
    163 216 242 281 312
    160 207 248 288 324
    142 187 234 280 316
    156 203 243 283 317
    157 212 259 307 336
    152 203 246 286 321
    154 205 253 298 334
    139 190 225 267 302
    146 191 229 272 302
    157 211 250 285 323
    132 185 237 286 331
    160 207 257 303 345
    169 216 261 295 333
    157 205 248 289 316
    137 180 219 258 291
    153 200 244 286 324
]

data = Dict(:x => x, :xbar => xbar, :Y => Y, :N => N, :T => T)

# initializations
alpha = ones(Integer, 30) .* 250
beta = ones(Integer, 30) .* 6
alpha_c = 150
beta_c = 10
tau_c = 1
alpha_tau = 1
beta_tau = 1

initializations = Dict(
    :alpha => alpha,
    :beta => beta,
    :alpha_c => alpha_c,
    :beta_c => beta_c,
    :tau_c => tau_c,
    :alpha_tau => alpha_tau,
    :beta_tau => beta_tau,
);

initializations_2 = Dict(
    :alpha => alpha ./ 10,
    :beta => beta ./ 10,
    :alpha_c => alpha_c / 10,
    :beta_c => beta_c /10,
    :tau_c => tau_c / 10,
    :alpha_tau => alpha_tau / 10,
    :beta_tau => beta_tau / 10,
)

Dict{Symbol, Any} with 7 entries:
  :alpha     => [25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0  … …
  :beta      => [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6  …  0.6, 0.6,…
  :beta_c    => 1.0
  :tau_c     => 0.1
  :alpha_tau => 0.1
  :beta_tau  => 0.1
  :alpha_c   => 15.0

In [3]:
vars, array_map, var_types = program!(CollectVariables(), model_def, data);
dep_graph = program!(DependencyGraph(vars, array_map), model_def, data);
node_args, node_functions, link_functions = program!(NodeFunctions(vars, array_map), model_def, data);

In [13]:
# m = JuliaBUGS.BUGSModel(vars, array_map, var_types, dep_graph, node_args, node_functions, link_functions, data, initializations);
m = JuliaBUGS.BUGSModel(vars, array_map, var_types, dep_graph, node_args, node_functions, link_functions, data, initializations_2);
trace = m.trace; # initial trace, value copmuted according to data and initializations
parameters = m.parameters; # parameter is a Vector{Var}

In [14]:
# struct BUGSLogDensityProblem 
#     m::BUGSModel
# end
p = JuliaBUGS.BUGSLogDensityProblem(m);

In [15]:
initial_θ = JuliaBUGS.transform_and_flatten(trace, parameters, m.bijectors)

65-element Vector{Float64}:
 -2.3025850929940455
 25.0
 25.0
 25.0
 25.0
 25.0
 25.0
 25.0
 25.0
 25.0
 25.0
 25.0
 25.0
  ⋮
  0.6
  0.6
  0.6
  0.6
  0.6
  0.6
  0.6
  0.6
  0.6
  0.6
  0.6
  1.0

In [16]:
p(initial_θ) # compute log_joint

-380769.7837397601

In [17]:
using ReverseDiff
g = ReverseDiff.gradient(p, initial_θ)

65-element Vector{Float64}:
 -380081.2991
     106.4
     110.5
     112.9
     102.60000000000001
     102.10000000000002
     111.50000000000001
     100.60000000000002
     110.80000000000001
     128.90000000000003
      95.7
     115.9
     100.30000000000001
       ⋮
     265.3400000000001
     287.7400000000002
     252.74000000000004
     245.74
     254.84000000000015
     319.9400000000001
     296.84000000000015
     255.54000000000008
     252.0400000000001
     240.84000000000012
     270.24000000000007
      -1.2000010000000008

In [9]:
println("Variable", "  Initial_value", "  Gradient  ")
for i in 1:65
    println(parameters[i], "  ", initial_θ[i], "  ", g[i])
end

Variable  Initial_value  Gradient  
tau_c  0.0  -23486.0
alpha[1]  250.0  -151.0
alpha[2]  250.0  -110.0
alpha[3]  250.0  -86.0
alpha[4]  250.0  -189.0
alpha[5]  250.0  -194.0
alpha[6]  250.0  -100.0
alpha[7]  250.0  -209.0
alpha[8]  250.0  -107.0
alpha[9]  250.0  74.0
alpha[10]  250.0  -258.0
alpha[11]  250.0  -56.0
alpha[12]  250.0  -212.0
alpha[13]  250.0  -138.0
alpha[14]  250.0  -4.0
alpha[15]  250.0  -136.0
alpha[16]  250.0  -123.0
alpha[17]  250.0  -191.0
alpha[18]  250.0  -148.0
alpha[19]  250.0  -79.0
alpha[20]  250.0  -142.0
alpha[21]  250.0  -106.0
alpha[22]  250.0  -227.0
alpha[23]  250.0  -210.0
alpha[24]  250.0  -124.0
alpha[25]  250.0  -179.0
alpha[26]  250.0  -78.0
alpha[27]  250.0  -76.0
alpha[28]  250.0  -135.0
alpha[29]  250.0  -265.0
alpha[30]  250.0  -143.0
beta_tau  0.0  -226.0
alpha_tau  0.0  -149986.0
alpha_c  150.0  2999.99985
beta[2]  6.0  648.0
beta[1]  6.0  18.0
beta[3]  6.0  284.0
beta[4]  6.0  -444.0
beta[5]  6.0  340.0
beta[6]  6.0  88.0
beta[7]  6.0  -38

In [18]:
using AdvancedHMC
using ReverseDiff

D = length(initial_θ)
n_samples, n_adapts = 2000, 1000

metric = DiagEuclideanMetric(D)
hamiltonian = Hamiltonian(metric, p, :ReverseDiff)

initial_ϵ = find_good_stepsize(hamiltonian, initial_θ)
integrator = Leapfrog(initial_ϵ)
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; drop_warmup=true, progress=true);

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Sampling 100%|███████████████████████████████| Time: 0:13:17
  iterations:                                   2000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9586021194371614
  log_density:                                  -651.510537827625
  hamiltonian_energy:                           677.585581667357
  hamiltonian_energy_error:                     -0.0559113310940802
  max_hamiltonia

In [19]:
beta_c_samples = [samples[s][64] for s in 1:length(samples)]
stats = mean(beta_c_samples), std(beta_c_samples) # Reference result: mean 6.186, variance 0.1088

(6.140193826602948, 0.2534543378959439)